In [2]:
import os
import pyodbc
import pandas as pd

In [3]:
os.listdir('./../../repos/LASOS-INT/TTBsim/01_Config/DBFiles/')

['AGFA.mdb',
 'Modelo de Dados - ToolBoxMix.png',
 'off',
 'off2',
 'ToolBoxMix_Development - Faz Tudo Quero Logo.mdb',
 'ToolBoxMix_Development.ldb',
 'ToolBoxMix_Development.mdb',
 'ToolBoxMix_InjecaoPlastica.mdb',
 'ToolBoxMix_MetalMeca - backup.mdb',
 'ToolBoxMix_MetalMeca.mdb']

In [4]:
path = './../../repos/LASOS-INT/TTBsim/01_Config/DBFiles/ToolBoxMix_MetalMeca.mdb'
conn_str =     conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};' +
            'DBQ='+f'{path};')
print(conn_str)
conn = pyodbc.connect(conn_str)

DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=./../../repos/LASOS-INT/TTBsim/01_Config/DBFiles/ToolBoxMix_MetalMeca.mdb;


In [5]:
cursor = conn.cursor()
for i in cursor.tables(tableType='TABLE'):
    print(i.table_name)

ACABADO
AGRU_ENTID
AGRU_GRUPO
AGRU_LINK
ALTERNATIVAS
ATIV_CENTRO
ATIV_PRECED
ATIVP
CAL_EXCE
CAL_PART
COMPRADO
CONJUGADAS
CONJUNTOS
CTRL_APONTA
CTRL_ESTOQ_DEPO
CTRL_ESTOQ_MOV
CTRL_ESTOQ_POS
CTRL_MARCO
DEC_CRITERIOS_DE_SEQUENCIAMENTO
DEC_DEDICACAO_MAQUINAS
DEC_FLEXIBILIDADES
DEC_HORA_EXTRA
DEC_HORARIOS
DEC_LOTES_PRODUCAO
DEC_LOTES_TRANSFERENCIA
DEC_PLANO_COMPRAS
DEC_PRAZOS_ENTREGAS
DEC_PRIORIDADE
DEC_RESTRICAO_INICIO
DEC_SIMULAR_ORDEM_PRODUCAO
DEC_SPLITS
DEC_SUBCONTRATACOES
DEC_TEMPO_TRANS
ELO_PERDIDO
Erros ao colar
ESCALARES
ESTOQUES
ESTOQUES_PROCESSO
EXT_COMPRA
EXT_COMPRA_RECEB
EXT_EMPRESA
EXT_PLANO_MESTRE
EXT_VENDA
EXT_VENDA_ENTREGA
EXT_VENDA_STATUS
FABRICA
FABRICA_OLD
FERRAMENTA
FLEX_ESCOPO_MAQ
FLEX_ESCOPO_OPER
HORARIOS
HORARIOS_PARTICAO
ITEM
ITEM_ESTRU
ITENS_NAVEGACAO
ITENS_SELECAO
LISTA_APLICACOES
LISTA_APLICACOES_AUX
MAQUINA
OPERACOES
REL_ANALISE_FINANCEIRA
REL_DEBUG
REL_DEMANDAS_LIQUIDAS
REL_ENTREGAS_NAO_REALIZADAS
REL_NECESS_MAT
REL_PONTUALIDADE
REL_RECEB_MAT
REL_SEQUENCIA_DETAL

In [13]:
TAREFAS = pd.read_sql('select * from REL_TAREFAS', conn)
TAREFAS.sort_values(by=['CdAtiv', 'NuEstagio', 'CdMaq'], inplace=True)
TAREFAS

C:\Users\arthur.silva\AppData\Local\Temp\ipykernel_24948\2489382701.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  TAREFAS = pd.read_sql('select * from REL_TAREFAS', conn)


,IdTar,IdTarAgreg,CdMaq,CdFerr,CdEntrega,CdAcabado,DtEntrega,CdAtiv,CdItem,NuEstagio,...,TmRepouso,TmTrans,TmLiqSetup,TmLiqProc,TmLiqRepouso,TmLiqTrans,QtProg,NumCriterio,FgAtrasou,FgEntrega
22,22,22,020,-,Entrega1,None,None,OS/120#1,None,None,...,0.0,0.0,0.0,57600.0,0.0,0.0,4.0,5,False,False
36,36,36,013,-,Entrega1,None,None,OS/120#2,None,None,...,0.0,0.0,0.0,28800.0,0.0,0.0,4.0,5,False,False
45,45,45,006,-,Entrega1,None,None,OS/120#3,None,None,...,0.0,0.0,0.0,43200.0,0.0,0.0,4.0,5,False,False
52,52,52,011,-,Entrega1,None,None,OS/120#4,None,None,...,0.0,0.0,0.0,72000.0,0.0,0.0,4.0,-1,False,False
80,80,80,010,-,Entrega1,None,None,OS/120#5,None,None,...,0.0,0.0,0.0,100800.0,0.0,0.0,4.0,5,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,233,233,006,-,Entrega41,None,None,OS/160#3,None,None,...,0.0,0.0,0.0,10800.0,0.0,0.0,1.0,-1,False,False
257,257,257,012,-,Entrega41,None,None,OS/160#4,None,None,...,0.0,0.0,0.0,18000.0,0.0,0.0,1.0,-1,False,False
273,273,273,010,-,Entrega41,None,None,OS/160#5,None,None,...,0.0,0.0,0.0,25200.0,0.0,0.0,1.0,-1,False,False
277,277,277,014,-,Entrega41,None,None,OS/160#6,None,None,...,0.0,0.0,0.0,18000.0,0.0,0.0,1.0,-1,False,False
